In [7]:
!pip install pytest beautifulsoup4 lxml selenium

  Obtaining dependency information for selenium from https://files.pythonhosted.org/packages/aa/85/fa44f23dd5d5066a72f7c4304cce4b5ff9a6e7fd92431a48b2c63fbf63ec/selenium-4.25.0-py3-none-any.whl.metadata
  Obtaining dependency information for trio~=0.17 from https://files.pythonhosted.org/packages/1c/70/efa56ce2271c44a7f4f43533a0477e6854a0948e9f7b76491de1fd3be7c9/trio-0.26.2-py3-none-any.whl.metadata
  Obtaining dependency information for trio-websocket~=0.9 from https://files.pythonhosted.org/packages/48/be/a9ae5f50cad5b6f85bd2574c2c923730098530096e170c1ce7452394d7aa/trio_websocket-0.11.1-py3-none-any.whl.metadata
  Obtaining dependency information for typing_extensions~=4.9 from https://files.pythonhosted.org/packages/26/9f/ad63fc0248c5379346306f8668cda6e2e2e9c95e01216d2b8ffd9ff037d0/typing_extensions-4.12.2-py3-none-any.whl.metadata
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
  Obtaining dependency information for websocket-client~=1.8 from https://files

In [9]:
import urllib.request
from bs4 import BeautifulSoup
from datetime import datetime

# Hàm lấy danh sách bài viết từ trang chủ
def collect_articles_from_page(page_url, key='h3'):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:11.0) Gecko/20100101'}
    request = urllib.request.Request(page_url, headers=headers)
    
    response = urllib.request.urlopen(request)
    page_html = response.read()
    
    soup = BeautifulSoup(page_html, 'html.parser')
    
    # Tìm tất cả các thẻ <h3> hoặc thẻ khác chứa link bài báo
    article_tags = soup.find_all('h3') or soup.find_all('article')  # Thử các thẻ khác nhau nếu cần
    
    article_urls = []
    for tag in article_tags:
        link = tag.find('a', href=True)
        if link:
            href = link['href']
            if href.startswith('/'):
                href = 'https://vnexpress.net' + href
            article_urls.append(href)
    
    return article_urls

# Hàm lấy chi tiết từ từng bài báo
def extract_article_details(article_url):
    try:
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:11.0) Gecko/20100101'}
        request = urllib.request.Request(article_url, headers=headers)
        
        # Gửi yêu cầu lấy trang bài báo
        response = urllib.request.urlopen(request)
        article_html = response.read()
        
        # Parse bài báo
        soup = BeautifulSoup(article_html, 'html.parser')
        
        # Lấy tiêu đề
        title = soup.find('h1', class_='title-detail').text.strip() if soup.find('h1', class_='title-detail') else None
        
        # Lấy nội dung
        content_paragraphs = soup.find_all('p')
        content = ""
        for para in content_paragraphs:
            content += para.get_text(separator=" ") + "\n"
        
        # Lấy ngày đăng bài
        published_date = soup.find('span', class_='date').text.strip() if soup.find('span', class_='date') else None
        
        # Luôn lấy dòng cuối cùng làm tên tác giả
        author_info = "Unknown"
        if len(content) > 0:
            # Lấy dòng cuối cùng
            content_lines = content.strip().split("\n")
            last_line = content_lines[-1].strip()
            
            # Xóa ký tự không mong muốn ("\n", ",", ".")
            author_info = last_line.replace("\n", "").replace(",", "").replace(".", "").strip()
            content = content.rsplit('.', 1)[0].strip()  # Xóa câu cuối cùng chứa tên tác giả
        
        # Ngày thu thập dữ liệu
        collected_date = datetime.now().strftime('%Y-%m-%d')
        
        return {
            "title": title,
            "url": article_url,
            "content": content,
            "published_date": published_date,
            "author_info": author_info,
            "collected_date": collected_date
        }
    except Exception as e:
        print(f"Could not retrieve article {article_url}: {e}")
        return None

# Lặp qua các trang từ 1 đến 5
for page_num in range(1, 10):
    if page_num == 1:
        page_url = 'https://vnexpress.net/kinh-doanh'
    else:
        page_url = f'https://vnexpress.net/kinh-doanh-p{page_num}'
    
    print(f"Collecting articles from: {page_url}")
    
    # Lấy tất cả URL bài viết từ trang hiện tại
    article_urls = collect_articles_from_page(page_url)
    
    # Duyệt qua từng bài báo và lấy thông tin chi tiết
    for article_url in article_urls:
        article_details = extract_article_details(article_url)
        
        # Nếu lấy được thông tin chi tiết, in ra
        if article_details:
            print(article_details)
        else:
            print(f"Skipping article: {article_url}")

{'title': 'USD có tuần tăng mạnh nhất 2 năm', 'url': 'https://vnexpress.net/usd-co-tuan-tang-manh-nhat-2-nam-4800454.html', 'content': 'Căng thẳng tại Trung Đông cùng báo cáo việc làm Mỹ tháng 9 tốt hơn kỳ vọng khiến Dollar Index tăng 2,1% tuần này - mạnh nhất 2 năm.\nChốt phiên 4/10, Dollar Index - đo sức mạnh của đồng bạc xanh với rổ tiền tên lớn - tăng 0,5% lên 102,49 điểm. Đây là mức cao nhất kể từ giữa tháng 8. Tính chung cả tuần, chỉ số này tăng 2,1% - tốt nhất kể từ tháng 9/2022.\nUSD tăng giá sau khi Mỹ hôm qua công bố báo cáo việc làm tháng 9 lạc quan. Nền kinh tế lớn nhất thế giới đã tạo thêm 254.000 việc làm mới, cao hơn nhiều so với dự báo 140.000 mà các nhà kinh tế học đưa ra trong khảo sát trước đó của Reuters. Tỷ lệ thất nghiệp cũng bất ngờ giảm, từ 4,2% tháng 8 về 4,1% tháng 9.\nDiễn biến Dollar Index trong 3 tháng qua. Đồ thị:  MarketWatch\n"Báo cáo việc làm rất tốt. Kịch bản Mỹ vẫn tăng trưởng bất chấp lãi suất ở mức cao đang khả thi hơn bao giờ hết", Karl Schamotta -

In [4]:
import json


# Save to JSON function
def save_to_json(data, filename="articles.json"):
    with open(filename, 'w', encoding='utf-8') as file:
        json.dump(data, file, ensure_ascii=False, indent=4)

# Main function to run the scraping process and save to JSON
def main():
    collected_articles = []
    
    for page_num in range(1, 6):
        if page_num == 1:
            page_url = 'https://vnexpress.net/kinh-doanh'
        else:
            page_url = f'https://vnexpress.net/kinh-doanh-p{page_num}'
        
        print(f"Collecting articles from: {page_url}")
        article_urls = collect_articles_from_page(page_url)
        
        for article_url in article_urls:
            article_details = extract_article_details(article_url)
            if article_details:
                collected_articles.append(article_details)
    
    # Save the collected articles to JSON
    save_to_json(collected_articles)

if __name__ == "__main__":
    main()

In [11]:
import urllib.request
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

# Hàm chuyển đổi datetime sang Unix timestamp
def date_to_timestamp(date_obj):
    return int(date_obj.timestamp())

# Hàm lấy danh sách bài viết từ trang chủ
def collect_articles_from_page(page_url):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:11.0) Gecko/20100101'}
    request = urllib.request.Request(page_url, headers=headers)
    
    response = urllib.request.urlopen(request)
    page_html = response.read()
    
    soup = BeautifulSoup(page_html, 'html.parser')
    
    article_tags = soup.find_all('h3') or soup.find_all('article')
    
    article_urls = []
    for tag in article_tags:
        link = tag.find('a', href=True)
        if link:
            href = link['href']
            if href.startswith('/'):
                href = 'https://vnexpress.net' + href
            article_urls.append(href)
    
    return article_urls

# Hàm lấy chi tiết từ từng bài báo
def extract_article_details(article_url):
    try:
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:11.0) Gecko/20100101'}
        request = urllib.request.Request(article_url, headers=headers)
        
        response = urllib.request.urlopen(request)
        article_html = response.read()
        
        soup = BeautifulSoup(article_html, 'html.parser')
        
        title = soup.find('h1', class_='title-detail').text.strip() if soup.find('h1', class_='title-detail') else None
        
        content_paragraphs = soup.find_all('p')
        content = ""
        for para in content_paragraphs:
            content += para.get_text(separator=" ") + "\n"
        
        published_date = soup.find('span', class_='date').text.strip() if soup.find('span', class_='date') else None
        
        author_info = "Unknown"
        if len(content) > 0:
            content_lines = content.strip().split("\n")
            last_line = content_lines[-1].strip()
            author_info = last_line.replace("\n", "").replace(",", "").replace(".", "").strip()
            content = content.rsplit('.', 1)[0].strip()
        
        collected_date = datetime.now().strftime('%Y-%m-%d')
        
        return {
            "title": title,
            "url": article_url,
            "content": content,
            "published_date": published_date,
            "author_info": author_info,
            "collected_date": collected_date
        }
    except Exception as e:
        print(f"Could not retrieve article {article_url}: {e}")
        return None

# Hàm để tạo các URL cho khoảng thời gian
def generate_urls_for_date_range(start_date, end_date):
    urls = []
    current_date = start_date
    while current_date <= end_date:
        timestamp = date_to_timestamp(current_date)
        url = f'https://vnexpress.net/category/day/cateid/1003159/fromdate/{timestamp}/todate/{timestamp}/allcate/1003159'
        urls.append(url)
        current_date += timedelta(days=1)
    return urls

# Tạo các URL cho khoảng từ 4-9-2024 đến 5-10-2024
start_date = datetime(2024, 10, 4)
end_date = datetime(2024, 10, 5)
urls_by_date = generate_urls_for_date_range(start_date, end_date)

# Lặp qua từng URL để thu thập bài viết
for page_url in urls_by_date:
    print(f"Collecting articles from: {page_url}")
    
    # Lấy tất cả URL bài viết từ trang hiện tại
    article_urls = collect_articles_from_page(page_url)
    
    # Duyệt qua từng bài báo và lấy thông tin chi tiết
    for article_url in article_urls:
        article_details = extract_article_details(article_url)
        
        # Nếu lấy được thông tin chi tiết, in ra
        if article_details:
            print(article_details)
        else:
            print(f"Skipping article: {article_url}")


{'title': "Wipro Vietnam tiếp tục được vinh danh 'Doanh nghiệp xuất sắc châu Á'", 'url': 'https://vnexpress.net/wipro-vietnam-tiep-tuc-duoc-vinh-danh-doanh-nghiep-xuat-sac-chau-a-4800359.html', 'content': 'TP HCM Wipro Consumer Care Vietnam lần thứ hai liên tiếp được vinh danh trong hạng mục "Doanh nghiệp xuất sắc châu Á" tại giải thưởng Asia Pacific Enterprise Awards (APEA) 2024.\nVượt qua hàng trăm doanh nghiệp tham gia đề cử, Wipro Consumer Care Vietnam tiếp tục lọt vào danh sách "Doanh nghiệp xuất sắc châu Á" của APEA 2024 nhờ thành tựu trong kinh doanh cũng như đóng góp cho xã hội. Các ứng viên cho danh hiệu này đã phải trải qua quy trình đánh giá với nhiều tiêu chí và nhiều vòng xét duyệt, từ hồ sơ doanh nghiệp đến phỏng vấn trực tiếp. Giải thưởng là cơ hội để các doanh nghiệp nâng cao vị thế, khẳng định và lan tỏa độ nhận diện thương hiệu của mình ở khu vực châu Á cũng như trên thị trường quốc tế.\nĐại diện Wipro Consumer Care Vietnam (trái) nhận giải thưởng Doanh nghiệp xuất sắ